## 今天的作業
在鐵達尼資料集中，以Titanic_train.csv 中，首先將有遺失值的數值刪除，試著將課堂中所學的方法應用上去。
* Q1: 目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。


In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
print(pd.__version__)
%matplotlib inline




# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

 

0.24.2
1.1.3


## 讀入資料

In [2]:
path="C:/Users/ADMIN/"
df_train = pd.read_csv(path+"Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
#1.產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態      
#2.把題目中的 Survived 用 Survived_cate 來做分析    
df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [4]:
#我們先把遺失值刪除
## 取出資料後，把遺失值刪除
complete_data=df_train.dropna()

#排除 Ｎame 當特徵，先刪除每一個人都是獨特的資料，先檢視其他變數
complete_data=complete_data.drop(['Name','Ticket','PassengerId'], axis=1)
display(complete_data.head(5))

print(complete_data.shape)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate
1,1,1,female,38.0,1,0,71.2833,C85,C,1
3,1,1,female,35.0,1,0,53.1000,C123,S,1
6,0,1,male,54.0,0,0,51.8625,E46,S,0
10,1,3,female,4.0,1,1,16.7000,G6,S,1
11,1,1,female,58.0,0,0,26.5500,C103,S,1


(183, 10)


In [5]:
num_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

6 Numeric Features : ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [6]:
cat_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'object':
        cat_features.append(feature)
print(f'{len(cat_features)} category Features : {cat_features}\n')

4 category Features : ['Sex', 'Cabin', 'Embarked', 'Survived_cate']



## 定義 x,y

In [7]:
x=complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']]
y=complete_data['Survived']
display(x.head(5))
display(y.head(5))

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,female,C
3,1,35.0,1,0,53.1000,female,S
6,1,54.0,0,0,51.8625,male,S
10,3,4.0,1,1,16.7000,female,S
11,1,58.0,0,0,26.5500,female,S


1     1
3     1
6     0
10    1
11    1
Name: Survived, dtype: int64

### Q1: 目標變數為 Survived，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性!


In [15]:
#離散要轉換成數值，['Sex', 'Embarked']
#離散型資料+連續
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile, chi2
##ValueError: could not convert string to float: 'Male'
#離散型資料要先轉成數值


sex_mapping = {
           'male': 1,
           'female': 0}
complete_data['Sex1'] = complete_data['Sex'].map(sex_mapping)

Embarked_mapping = {
           'C': 0,
           'Q': 1,
           'S': 2}
complete_data['Embarked1'] = complete_data['Embarked'].map(Embarked_mapping)

x=complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex1', 'Embarked1']]
display(x)
y=complete_data['Survived']
x_new = SelectPercentile(chi2, percentile=50).fit_transform(x, y)
x_new.shape
display(x_new)

,Pclass,Age,SibSp,Parch,Fare,Sex1,Embarked1
1,1,38.0,1,0,71.2833,0,0
3,1,35.0,1,0,53.1000,0,2
6,1,54.0,0,0,51.8625,1,2
10,3,4.0,1,1,16.7000,0,2
11,1,58.0,0,0,26.5500,0,2
...,...,...,...,...,...,...,...
871,1,47.0,1,1,52.5542,0,2
872,1,33.0,0,0,5.0000,1,2
879,1,56.0,0,1,83.1583,0,0
887,1,19.0,0,0,30.0000,0,2


array([[ 38.    ,  71.2833,   0.    ],
       [ 35.    ,  53.1   ,   0.    ],
       [ 54.    ,  51.8625,   1.    ],
       [  4.    ,  16.7   ,   0.    ],
       [ 58.    ,  26.55  ,   0.    ],
       [ 34.    ,  13.    ,   1.    ],
       [ 28.    ,  35.5   ,   1.    ],
       [ 19.    , 263.    ,   1.    ],
       [ 49.    ,  76.7292,   0.    ],
       [ 65.    ,  61.9792,   1.    ],
       [ 45.    ,  83.475 ,   1.    ],
       [ 29.    ,  10.5   ,   0.    ],
       [ 25.    ,   7.65  ,   1.    ],
       [ 23.    , 263.    ,   0.    ],
       [ 46.    ,  61.175 ,   1.    ],
       [ 71.    ,  34.6542,   1.    ],
       [ 23.    ,  63.3583,   1.    ],
       [ 21.    ,  77.2875,   1.    ],
       [ 47.    ,  52.    ,   1.    ],
       [ 24.    , 247.5208,   1.    ],
       [ 32.5   ,  13.    ,   0.    ],
       [ 54.    ,  77.2875,   1.    ],
       [ 19.    ,  26.2833,   0.    ],
       [ 37.    ,  53.1   ,   1.    ],
       [ 24.    ,  79.2   ,   1.    ],
       [ 36.5   ,  26.   

In [16]:
#執行 包裝法
estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(x, y)
## True= selected feature
print(selector.support_)
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)


[False False  True False False  True False]
[2 4 1 5 6 1 3]
['SibSp', 'Sex1']
